In [ ]:
!pip install ultralytics
!pip install roboflow
!pip install python-dotenv

## Data set of Roboflow

The data set used for this train is found in the following [link to Roboflow](https://universe.roboflow.com/fashion-g3m22/type-k4lkg)

#### Dataset Split

|  Train set  | Valid set  |  Test set  |
|:-----------:|:----------:|:----------:|
| 6037 images | 682 images | 742 images |
|     81%     |     9%     |    10%     |

#### Preprocessing

- Auto-Orient: Applied
- Resize: Stretch to 640x640

#### Download structure

Folder Structure

#### Classes in this data set

- dress
- hat
- longsleeve 
- outwear 
- pants
- shirt
- shoes
- shorts
- skirt
- t-shirt

In [ ]:
from roboflow import Roboflow
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.environ.get('ROBOFLOW_API_KEY')

rf = Roboflow(api_key=api_key)
project = rf.workspace("fashion-g3m22").project("type-k4lkg")
version = project.version(1)
dataset = version.download("folder")


For this train I use the YOLOv8s-cls

| Model                                                                                        | size (pixels) | acc top1 | acc top5 | Speed CPU ONNX (ms) | Speed A100 TensorRT (ms) | params (M) | FLOPs (B) at 640 |
|:-------------------------------------------------------------------------------------------- |:-------------:|:--------:|:--------:|:-------------------:|:------------------------:|:----------:|:----------------:|
| YOLOv8n-cls                                                                                  |      224      |   69.0   |   88.3   |        12.9         |           0.31           |    2.7     |       4.3        |
| [YOLOv8s-cls](https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov8s-cls.pt) |      224      |   73.8   |   91.7   |        23.4         |           0.35           |    6.4     |       13.5       |
| YOLOv8m-cls                                                                                  |      224      |   76.8   |   93.5   |        85.4         |           0.62           |    17.0    |       42.7       |
| YOLOv8l-cls                                                                                  |      224      |   76.8   |   93.5   |        163.0        |           0.87           |    37.5    |       99.7       |
| YOLOv8x-cls                                                                                  |      224      |   79.0   |   94.6   |        232.0        |           1.01           |    57.4    |      154.8       |


In [ ]:
from ultralytics import YOLO

In [ ]:
# Load a model
model = YOLO('yolov8s-cls.pt')  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data='type-1', epochs=55, imgsz=640)

In [ ]:
# Load a new custom model
model = YOLO('runs/classify/train/weights/best.pt')

##### Original image
![original image](pants.png)

In [ ]:
results = model('pants.png', verbose=True)  # predict on an pants image

image 1/1 pants.png: 640x640 pants 0.93, shorts 0.04, shirt 0.01, longsleeve 0.01, skirt 0.01, 423.7ms
Speed: 74.1ms preprocess, 423.7ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
for r in results:
    r.save(filename="results.png")

##### Image with predicts
![image with predicts](results.png)

# Colors processing

## Recognize predominant color function

This function `recognize_predominant_color` will open the PNG image, convert the pixels to RGBA, and then iterate over them, ignoring pixels that have an alpha (transparency) channel of 0. It will then count the frequency of each non-transparent color and return the predominant color.

In [ ]:
from PIL import Image
from collections import Counter


def recognize_predominant_color(image_path):
    img = Image.open(image_path)

    # Convert the image to mode RGBA
    img = img.convert("RGBA")

    # Get the pixels of teh image
    pixels = img.getdata()

    # Filter transparent pixels
    pixels_without_transparency = [p[:3] for p in pixels if p[3] != 0]

    # Count the frequency of each color
    counter = Counter(pixels_without_transparency)

    # Get the predominant color
    color = counter.most_common(1)[0][0]

    return color


# Ruta de la imagen
image = "pants.png"
# Llamar a la función y obtener el color predominante
color_predominant = recognize_predominant_color(image)
print("The color predominant is:", color_predominant)

The color predominant is: (30, 45, 78)


**The color of the pants is also:**

![pants color](media/pants_color.png)

## Matching Colors Function

This function `matching_colors` takes a selected color and a list of colors as input, along with a tolerance value. It then returns a list of colors from the input list that match well with the selected color within the specified tolerance range.

### Usage:

- **`selected_color`**: The RGB color tuple that you want to find matches for.
- **`color_list`**: A list of RGB color tuples.
- **`tolerance`**: A numerical value representing the maximum allowable difference between the selected color and the colors in the list for them to be considered a match.

In [ ]:
def calculate_color_difference(color1, color2):
    """
    Calculates the difference between two colors in RGB color space.
    Returns the total difference.
    """
    return sum(abs(component1 - component2) for component1, component2 in zip(color1, color2))


def color_combines(color_base, color_to_compare, tolerance):
    """
    Checks if a given color combines with another within a tolerance range.
    """
    difference = calculate_color_difference(color_base, color_to_compare)
    return difference <= tolerance


def matching_colors(selected_color, color_list, tolerance):
    """
    Returns a list of colors that match with the selected color within the given list,
    with a given tolerance.
    """
    matching_colors = []
    for color in color_list:
        if color_combines(selected_color, color, tolerance):
            matching_colors.append(color)
    return matching_colors


# Suppose you have a list of colors
color_list = [
    (100, 50, 30),
    (120, 70, 40),
    (90, 40, 20),
    (140, 80, 50),
    # You can add more colors here
]

# Suppose you have selected a color from your list
selected_color = (150, 100, 50)

# Define a tolerance
tolerance = 50  # You can adjust this value according to your needs

# Get the colors that match with the selected color within the list
matching_colors = matching_colors(selected_color, color_list, tolerance)

# Print the matching colors
print("Colors that match with the selected color:")
for color in matching_colors:
    print(color)